<a href="https://colab.research.google.com/github/DebStar17/2-Piece-Chess/blob/main/Copy_of_2_pieceChess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# P1 (Knight) and P2 (King) Movements

In [132]:
#Define p1 and p2 movements
movement_p1 = [[2, -1], [2, 1], [1, -2], [1, 2], [-2, -1], [-2, 1], [-1, -2], [-1, 2]]
movement_p2 = [[0, 1], [0, -1], [1, 0], [-1, 0], [-1, -1], [-1, 1], [1, 1], [1, -1]]

# All functions

In [133]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import ListedColormap
import random

### Show Grid

In [134]:
import numpy as np
import matplotlib.pyplot as plt

def show_grid(grid):
  grid = np.array(grid)
  rows, cols = grid.shape

  # 1. Define the color map as an explicit dictionary (Value: RGB Tuple)
  # The color values are given as (R, G, B) tuples, normalized to 0-1.
  color_map = {
      0: (1.0, 1.0, 1.0), # White
      1: (1.0, 0.0, 0.0), # Red
      2: (0.0, 0.0, 1.0), # Blue
      3: (0.0, 0.0, 0.0)  # Black
  }

  # 2. Create the 3D RGB array
  # We initialize an empty array of shape (rows, cols, 3) with float type
  rgb_grid = np.zeros((rows, cols, 3), dtype=float)

  # Iterate through the grid and assign RGB colors
  for i in range(rows):
    for j in range(cols):
      value = grid[i, j]
      # Use .get() for safety, defaulting to white if an unexpected value appears
      rgb_grid[i, j] = color_map.get(value, (1.0, 1.0, 1.0))

  # 3. Display the RGB array directly
  plt.imshow(rgb_grid, interpolation='nearest')
  plt.gca().invert_yaxis()
  #

  # --- Text Labeling and Gridlines (Same as before) ---
  for i in range(rows):
    for j in range(cols):
      cell_value = grid[i, j]
      text_to_display = ''
      text_color = 'black'

      if cell_value == 1:
        text_to_display = 'N'
        text_color = 'white'
      elif cell_value == 2:
        text_to_display = 'K'
        text_color = 'white'
      else:
        continue

      plt.text(j, i, text_to_display,
               ha='center', va='center',
               color=text_color, fontsize=10,
               fontweight='bold')

  # Setting minor tick locations and drawing gridlines
  plt.gca().set_xticks(np.arange(-0.5, cols, 1), minor=True)
  plt.gca().set_yticks(np.arange(-0.5, rows, 1), minor=True)
  plt.grid(which='minor', color='black', linestyle='-', linewidth=2)
  plt.show()

### Surrounding Cells

In [135]:
def surr(p1x, p1y, p1, grid):
  gr = len(grid)
  surrli = []
  for x, y in p1:
    if -1 < p1x + x < gr and -1 < p1y + y < gr and grid[p1y + y][p1x + x] != 4:
      surrli.append((p1x + x, p1y + y))
  return surrli

### Play function

In [136]:
import random
def trained_play(my_p, opp, grid, my_surr, all_games, player="p1"): # If training dataset exists - wherein, all_games is a list of lists
  if opp in my_surr:
    return opp #"my_p wins"

  if not all_games:
    return random.choice(my_surr)

  for *moves, win in all_games:
    for i in range(len(moves)):
      if player == "p1":
        if moves[i] == (my_p, opp) and (win[0] == 1 or win[0] == 2):
          return moves[i+1][0]
      if player == "p2":
        if moves[i] == (opp, my_p) and (win[1] == 1 or win[1] == 2):
          return moves[i+1][1]

  return random.choice(my_surr)

# Gameplay

In [137]:
def gameplay(all_games, grid2, p1, p2, play): # Modifies grid2 in-place
  if play%2 == 0:
    my_surr = surr(p1[0], p1[1], movement_p1, grid2)
    grid2[p1[1]][p1[0]] = 0

    p1 = trained_play(p1, p2, grid2, my_surr, all_games, player="p1")
    grid2[p1[1]][p1[0]] = 1

  else:
    my_surr = surr(p2[0], p2[1], movement_p2, grid2)
    grid2[p2[1]][p2[0]] = 0

    p2 = trained_play(p2, p1, grid2, my_surr, all_games, player="p2")
    grid2[p2[1]][p2[0]] = 2

  return (p1,p2)

In [138]:
import pickle
def gplay(grid, initial_p1, initial_p2, filename = "data.dat", show = 0):
  p1 = initial_p1
  p2 = initial_p2
  try:
    with open(filename, "rb") as f:
      all_games = pickle.load(f)
  except:
    all_games = []

  curr_game = []

  import copy
  grid2 = copy.deepcopy(grid)
  w = 0

  for play in range(50):
    p1, p2 = gameplay(all_games, grid2, p1, p2, play)
    if show:
      show_grid(grid2)
      input("Press Enter to play: ")
    curr_game.append((p1,p2))
    if p1 == p2:
      if play%2==0:
        # P1 wins
        curr_game.append((1,0))
        w = 1
        break
      else:
        # P2 wins
        curr_game.append((0,1))
        w = 1
        break

  if not w:
    curr_game.append((2,2))

  all_games.append(curr_game)

  with open(filename, "wb") as f:
    pickle.dump(all_games, f)

# Main

In [139]:
def main():
  # Create data.dat
  f = open("data.dat", "ab")
  f.close()

  # Make Grid
  gr = int(input("Enter the size of the grid: "))
  bias = list(map(int, input("Enter Bias for space and wall: ").split()))
  grid = [[0 for _ in range(gr)] for _ in range(gr)]

  # Adding Walls
  for i in grid:
    for j in range(len(i)):
      i[j] = random.choices([0, 4], weights=bias, k=1)[0]

  show_grid(grid)

  p1x, p1y = map(int, input("Enter pos of P1: ").split())
  p2x, p2y = map(int, input("Enter pos of P2: ").split())

  grid[p1y][p1x] = 1
  grid[p2y][p2x] = 2

  show_grid(grid)
  print(grid)

  thresh = int(input("Enter Training Threshold: "))
  for i in range(thresh):
    gplay(grid, (p1x, p1y), (p2x,p2y), filename = "data.dat", show = 0)

  print("\nTRAINING COMPLETE\n\n")

  gplay(grid, (p1x, p1y), (p2x,p2y), filename = "data.dat", show = 1)

# User Interface

In [ ]:
main()